In [108]:
# !pip install tavily

  DEPRECATION: Building 'tavily' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'tavily'. Discussion can be found at https://github.com/pypa/pip/issues/6334



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tavily: filename=tavily-1.1.0-py3-none-any.whl size=6143 sha256=4e6de138a299ded563cd5fa116b66b75a5f2e5cfec2ba87bbd5341500afef32a
  Stored in directory: c:\users\nakul\appdata\local\pip\cache\wheels\df\60\d8\883106a49d84c6a06cf0690728f44bca27a214ce10c628e28d
Successfully built tavily


In [3]:
from tavily import TavilyClient

In [4]:
import sqlite3
import pandas as pd
from pathlib import Path

 
BASE_DIR = Path().resolve()

# Paths
CSV_PATH = BASE_DIR / "video_game_sales_2024.csv"    
DB_PATH = BASE_DIR / "video_game_sales_data.db"

print("CSV Path:", CSV_PATH)
print("DB Path:", DB_PATH)


CSV Path: C:\Users\Nakul\video_game_sales_2024.csv
DB Path: C:\Users\Nakul\v_data.db


In [5]:
df = pd.read_csv(CSV_PATH)

print("Dataset Shape:", df.shape)
df.head()


Dataset Shape: (64016, 14)


,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14


In [6]:
# Standardize column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.columns


Index(['img', 'title', 'console', 'genre', 'publisher', 'developer',
       'critic_score', 'total_sales', 'na_sales', 'jp_sales', 'pal_sales',
       'other_sales', 'release_date', 'last_update'],
      dtype='object')

In [7]:
# Create SQLite connection
conn = sqlite3.connect(DB_PATH)

print("SQLite database created at:", DB_PATH)


SQLite database created at: C:\Users\Nakul\v_data.db


In [8]:
TABLE_NAME = "video_game_sales"

# Write DataFrame to SQLite
df.to_sql(TABLE_NAME, conn, if_exists="replace", index=False)

print(f"Table '{TABLE_NAME}' created successfully.")


Table 'video_game_sales' created successfully.


In [9]:
query = """
SELECT name
FROM sqlite_master
WHERE type='table';
"""

pd.read_sql(query, conn)


,name
0,video_game_sales


In [10]:
preview_query = f"""
SELECT *
FROM {TABLE_NAME}
LIMIT 5;
"""

pd.read_sql(preview_query, conn)


,img,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,None
1,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,None
3,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,None
4,/games/boxart/full_4990510AmericaFrontccc.jpg,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14


In [135]:
schema_query = f"""
PRAGMA table_info({TABLE_NAME});
"""

schema_df = pd.read_sql(schema_query, conn)
schema_df


,cid,name,type,notnull,dflt_value,pk
0,0,product_id,TEXT,0,None,0
1,1,product_name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,discounted_price,TEXT,0,None,0
4,4,actual_price,TEXT,0,None,0
5,5,discount_percentage,TEXT,0,None,0
6,6,rating,TEXT,0,None,0
7,7,rating_count,TEXT,0,None,0
8,8,about_product,TEXT,0,None,0
9,9,user_id,TEXT,0,None,0


# RAG IMPLEMENTATION

In [15]:
schema_text = "Table: video_game_sales\n\nColumns:\n"

for _, row in schema_df.iterrows():
    schema_text += f"- {row['name']} ({row['type']})\n"

schema_text


'Table: video_game_sales\n\nColumns:\n- img (TEXT)\n- title (TEXT)\n- console (TEXT)\n- genre (TEXT)\n- publisher (TEXT)\n- developer (TEXT)\n- critic_score (REAL)\n- total_sales (REAL)\n- na_sales (REAL)\n- jp_sales (REAL)\n- pal_sales (REAL)\n- other_sales (REAL)\n- release_date (TEXT)\n- last_update (TEXT)\n'

In [16]:
schema_definition = """
DATABASE SCHEMA DEFINITION

Table Name: video_game_sales

Description:
This table stores video game sales and metadata information for the year 2024.
Each row represents a unique game release on a specific console/platform.

Column Definitions:

1. img (TEXT)
   - URL or path to the game cover image
   - Used only for display purposes
   - Not used in KPI calculations

2. title (TEXT)
   - Name of the video game
   - A game title may appear multiple times across different consoles

3. console (TEXT)
   - Platform on which the game was released
   - Examples: PlayStation, Xbox, Nintendo Switch, PC
   - Used for platform-level performance analysis

4. genre (TEXT)
   - Category of the game
   - Examples: Action, RPG, Sports, Shooter, Adventure
   - Used for genre-based KPI aggregation

5. publisher (TEXT)
   - Company responsible for publishing the game
   - Used to analyze publisher-wise market performance

6. developer (TEXT)
   - Studio that developed the game
   - Used for studio-level success analysis

7. critic_score (REAL)
   - Average critic rating score
   - Higher values indicate better critical reception
   - Used for correlation analysis between reviews and sales

8. total_sales (REAL)
   - Total worldwide sales across all regions
   - Measured in millions of units
   - Primary metric for overall performance

9. na_sales (REAL)
   - Sales in North America region
   - Measured in millions

10. jp_sales (REAL)
    - Sales in Japan region
    - Measured in millions

11. pal_sales (REAL)
    - Sales in Europe / PAL regions
    - Measured in millions

12. other_sales (REAL)
    - Sales in other global regions
    - Measured in millions

13. release_date (TEXT)
    - Official game release date
    - Used for year-based and time-based analysis

14. last_update (TEXT)
    - Date when the sales data was last updated
    - Used for data freshness validation

Relationships & Notes:
- No explicit primary key is defined (read-only analytical dataset)
- Each record is treated independently for aggregation
- SQLite internally maintains row uniqueness using rowid
"""
print(schema_definition)



DATABASE SCHEMA DEFINITION

Table Name: video_game_sales

Description:
This table stores video game sales and metadata information for the year 2024.
Each row represents a unique game release on a specific console/platform.

Column Definitions:

1. img (TEXT)
   - URL or path to the game cover image
   - Used only for display purposes
   - Not used in KPI calculations

2. title (TEXT)
   - Name of the video game
   - A game title may appear multiple times across different consoles

3. console (TEXT)
   - Platform on which the game was released
   - Examples: PlayStation, Xbox, Nintendo Switch, PC
   - Used for platform-level performance analysis

4. genre (TEXT)
   - Category of the game
   - Examples: Action, RPG, Sports, Shooter, Adventure
   - Used for genre-based KPI aggregation

5. publisher (TEXT)
   - Company responsible for publishing the game
   - Used to analyze publisher-wise market performance

6. developer (TEXT)
   - Studio that developed the game
   - Used for studio-le

In [17]:
kpi_definitions = """
KPI DEFINITIONS

1. Total Sales KPI
   - Based on the 'total_sales' column
   - Represents total global units sold
   - Used as the default metric for performance comparison

2. Regional Sales KPIs
   - North America Sales → na_sales
   - Japan Sales → jp_sales
   - Europe / PAL Sales → pal_sales
   - Other Regions Sales → other_sales
   - Used for region-specific market analysis

3. Top Performing Games
   - Defined as games with highest total_sales
   - Ranking is done in descending order
   - Default limit is user-specified (e.g., Top 5, Top 10)

4. Platform Performance KPI
   - Aggregates total_sales grouped by console
   - Used to identify dominant gaming platforms

5. Genre Performance KPI
   - Aggregates total_sales grouped by genre
   - Used to identify most profitable game genres

6. Publisher Performance KPI
   - Aggregates total_sales grouped by publisher
   - Used to analyze publisher market dominance

7. Developer Success KPI
   - Aggregates total_sales grouped by developer
   - Used to evaluate development studio performance

8. Average Critic Score KPI
   - Computed as average of critic_score
   - Used to measure critical reception trends

9. Sales vs Critic Correlation KPI
   - Compares critic_score with total_sales
   - Used for insight generation (not SQL-only)

10. Yearly Performance KPI
    - Derived from release_date
    - Used to filter or group sales by year

11. Console-Genre Combination KPI
    - Measures genre popularity per console
    - Uses GROUP BY console, genre

12. Market Share KPI
    - Percentage contribution of a game/platform/genre
    - Calculated using total_sales / overall sales
"""
print(kpi_definitions)



KPI DEFINITIONS

1. Total Sales KPI
   - Based on the 'total_sales' column
   - Represents total global units sold
   - Used as the default metric for performance comparison

2. Regional Sales KPIs
   - North America Sales → na_sales
   - Japan Sales → jp_sales
   - Europe / PAL Sales → pal_sales
   - Other Regions Sales → other_sales
   - Used for region-specific market analysis

3. Top Performing Games
   - Defined as games with highest total_sales
   - Ranking is done in descending order
   - Default limit is user-specified (e.g., Top 5, Top 10)

4. Platform Performance KPI
   - Aggregates total_sales grouped by console
   - Used to identify dominant gaming platforms

5. Genre Performance KPI
   - Aggregates total_sales grouped by genre
   - Used to identify most profitable game genres

6. Publisher Performance KPI
   - Aggregates total_sales grouped by publisher
   - Used to analyze publisher market dominance

7. Developer Success KPI
   - Aggregates total_sales grouped by develop

In [18]:
business_rules = """
BUSINESS RULES & ASSUMPTIONS

1. Default Metrics
   - If the user does not specify a metric, total_sales is used
   - Sales values are assumed to be in millions

2. Interpretation of Keywords
   - "Top", "Best", "Highest" → ORDER BY total_sales DESC
   - "Lowest", "Worst" → ORDER BY total_sales ASC

3. Year Handling
   - If a year is mentioned, filter using release_date
   - If no year is mentioned, include all available records

4. Region Handling
   - If region is specified (e.g., Japan), use corresponding regional sales
   - If region is not specified, use total_sales

5. Platform & Genre Filters
   - Platform refers to the console column
   - Genre refers to the genre column

6. Aggregation Logic
   - Ranking queries require aggregation before sorting
   - Grouping is applied based on intent (genre, console, publisher)

7. Data Scope
   - Dataset is read-only
   - No INSERT, UPDATE, DELETE, or DROP operations allowed

8. Ambiguous Queries
   - If multiple interpretations are possible, prefer sales-based KPIs
   - Avoid assumptions beyond defined KPIs

9. Output Formatting
   - Results should be summarized in natural language
   - Raw SQL tables are converted to human-readable insights

10. Performance Considerations
    - Queries should limit results when ranking is involved
    - Aggregations should be optimized using GROUP BY
"""
print(business_rules)



BUSINESS RULES & ASSUMPTIONS

1. Default Metrics
   - If the user does not specify a metric, total_sales is used
   - Sales values are assumed to be in millions

2. Interpretation of Keywords
   - "Top", "Best", "Highest" → ORDER BY total_sales DESC
   - "Lowest", "Worst" → ORDER BY total_sales ASC

3. Year Handling
   - If a year is mentioned, filter using release_date
   - If no year is mentioned, include all available records

4. Region Handling
   - If region is specified (e.g., Japan), use corresponding regional sales
   - If region is not specified, use total_sales

5. Platform & Genre Filters
   - Platform refers to the console column
   - Genre refers to the genre column

6. Aggregation Logic
   - Ranking queries require aggregation before sorting
   - Grouping is applied based on intent (genre, console, publisher)

7. Data Scope
   - Dataset is read-only
   - No INSERT, UPDATE, DELETE, or DROP operations allowed

8. Ambiguous Queries
   - If multiple interpretations are possi

In [19]:
rag_documents = [
    {"type": "schema", "content": schema_definition},
    {"type": "kpi", "content": kpi_definitions},
    {"type": "business_rules", "content": business_rules}
]

print("RAG Knowledge Base Loaded:")
for doc in rag_documents:
    print(f"- {doc['type']}")


RAG Knowledge Base Loaded:
- schema
- kpi
- business_rules


## Part 3: Agentic RAG Implementation

This section implements an Agentic RAG architecture where multiple
specialized agents collaborate to convert natural language queries
into SQL and generate KPI insights.


In [33]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [70]:
import re

def clean_sql(sql_text: str) -> str:
    """
    Removes markdown code fences and extra text from LLM-generated SQL.
    """
    sql_text = re.sub(r"```sql", "", sql_text, flags=re.IGNORECASE)
    sql_text = re.sub(r"```", "", sql_text)
    return sql_text.strip()


In [110]:
tavily = TavilyClient(api_key=(""))


In [96]:
import google.generativeai as genai


genai.configure(api_key='')

# Initialize model
llm = genai.GenerativeModel("gemini-2.5-flash")

In [97]:
def call_gemini(prompt: str) -> str:
    response = llm.generate_content(prompt)
    return response.text.strip()


In [73]:
rag_texts = [doc["content"] for doc in rag_documents]

rag_embeddings = embedding_model.encode(
    rag_texts,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("RAG Embedding Shape:", rag_embeddings.shape)


RAG Embedding Shape: (3, 384)


In [74]:
dimension = rag_embeddings.shape[1]

index = faiss.IndexFlatIP(dimension) 
index.add(rag_embeddings)

print("FAISS index size:", index.ntotal)


FAISS index size: 3


In [75]:
id_to_doc = {
    i: rag_documents[i]
    for i in range(len(rag_documents))
}


In [76]:
def retrieve_rag_context(query: str, top_k: int = 3) -> str:
    query_embedding = embedding_model.encode(
        query,
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    scores, indices = index.search(
        np.array([query_embedding]), top_k
    )

    retrieved_chunks = []
    for idx in indices[0]:
        retrieved_chunks.append(id_to_doc[idx]["content"])

    return "\n\n".join(retrieved_chunks)


In [77]:
def query_understanding_agent(user_query: str):
    prompt = f"""
You are a query understanding agent.

Extract structured intent from the user query.

User Query:
"{user_query}"

Return output strictly in JSON with keys:
- task (ranking / aggregation)
- metric
- year (or null)
- limit (or null)
"""
    return call_gemini(prompt)


In [78]:
def rag_retrieval_agent(user_query: str):
    return retrieve_rag_context(user_query)


In [79]:
def kpi_reasoning_agent(intent_json, rag_context):
    prompt = f"""
You are a KPI reasoning agent.

User Intent (JSON):
{intent_json}

Use the following internal knowledge:
{rag_context}

Decide:
- Which column to use
- Whether aggregation is required
- Sorting logic
- Filtering logic

Return reasoning in structured bullet points.
"""
    return call_gemini(prompt)


In [85]:
def sql_generation_agent(intent_json, kpi_logic):
    prompt = f"""
You are a SQL generation agent.

DATABASE SCHEMA (STRICT — use ONLY these columns):
- img
- title
- console
- genre
- publisher
- developer
- critic_score
- total_sales
- na_sales
- jp_sales
- pal_sales
- other_sales
- release_date
- last_update

Table name: video_game_sales

User Intent:
{intent_json}

KPI Logic:
{kpi_logic}

RULES (MANDATORY):
- DO NOT invent column names
- Use 'title' for game name
- Read-only SQL
- SQLite compatible
- Output ONLY raw SQL (no markdown)

Generate the SQL now.
"""
    return call_gemini(prompt)


In [86]:
def execute_sql(sql):
    conn = sqlite3.connect(DB_PATH)
    df = pd.read_sql(sql, conn)
    conn.close()
    return df


In [87]:
def result_interpretation_agent(df, user_query):
    prompt = f"""
You are a data analyst agent.

User Question:
{user_query}

SQL Result:
{df.to_string(index=False)}

Explain the result in simple business language.
"""
    return call_gemini(prompt)


In [106]:
TAVILY_AGENT_PROMPT = """
ROLE:
You are a Tavily enrichment agent for a data analytics system.

PURPOSE:
Provide qualitative context for video game performance using web sources.

ALLOWED TASKS:
- Identify reported sales milestones (e.g., "sold over 10 million copies")
- Describe estimated popularity or market reception
- Summarize commercial success indicators such as:
  - Awards won
  - Critical acclaim
  - Player adoption
  - Market impact
  - Strong launch performance
  - Long-term sales reputation

STRICT RULES (MANDATORY):
- DO NOT invent or estimate numeric sales values
- DO NOT overwrite or modify database values
- DO NOT present estimated figures as confirmed data
- DO NOT claim precision when sources are vague
- Use cautious language such as:
  "reported to have sold over..."
  "widely considered a commercial success"
  "industry reports suggest strong sales performance"

OUTPUT FORMAT:
- Short bullet points per game
- Qualitative, non-numeric descriptions
- State that information is based on publicly available reports
"""


In [111]:
def tavily_enrichment_agent(game_title: str):
    query = f"""
{TAVILY_AGENT_PROMPT}

Game: {game_title}
"""
    result = tavily.search(query=query, max_results=5)
    return result.get("content", "")


In [113]:
def agentic_rag_pipeline(user_query: str):
    # 1️⃣ Query Understanding
    intent = query_understanding_agent(user_query)

    # 2️⃣ RAG Retrieval
    rag_context = retrieve_rag_context(user_query)

    # 3️⃣ KPI Reasoning
    kpi_logic = kpi_reasoning_agent(intent, rag_context)

    # 4️⃣ SQL Generation
    raw_sql = sql_generation_agent(intent, kpi_logic)
    cleaned_sql = clean_sql(raw_sql)

    # 5️⃣ SQL Execution
    sql_result = execute_sql(cleaned_sql)

    # 6️⃣ Result Interpretation (numeric explanation)
    analysis = result_interpretation_agent(sql_result, user_query)

    # 7️⃣ 🟢 CONDITIONAL Tavily Enrichment
    enrichment = {}

    for _, row in sql_result.iterrows():
        game_title = row["title"]
        total_sales = row["total_sales"]

        # 👉 condition: missing or zero sales
        if pd.isna(total_sales) or total_sales == 0:
            enrichment[game_title] = tavily_enrichment_agent(game_title)

    return {
        "intent": intent,
        "sql": cleaned_sql,
        "data": sql_result,
        "analysis": analysis,
        "enrichment": enrichment
    }


In [114]:
output = agentic_rag_pipeline("Top 5 games by sales")

print("SQL:\n", output["sql"])
print("\nAnalysis:\n", output["analysis"])
print("\nTavily Enrichment:\n", output["enrichment"])


SQL:
 SELECT title, SUM(total_sales) AS total_sales
FROM video_game_sales
GROUP BY title
ORDER BY total_sales DESC
LIMIT 5

Analysis:
 Here are the top 5 best-selling games by total sales:

1.  **Grand Theft Auto V** is the top performer, with sales reaching **64.29 million units**.
2.  **Call of Duty: Black Ops** comes in second with **30.99 million units** sold.
3.  **Call of Duty: Modern Warfare 3** is close behind, selling **30.71 million units**.
4.  **Call of Duty: Black Ops II** secured **29.59 million units** in sales.
5.  **Call of Duty: Ghosts** rounds out the top five with **28.80 million units** sold.

In summary, Grand Theft Auto V is a massive bestseller significantly ahead of others, and the Call of Duty franchise dominates the majority of the top spots, demonstrating its consistent popularity.

Tavily Enrichment:
 {}
